In [4]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator


In [11]:
load_dotenv()
model = ChatGoogleGenerativeAI(model = 'gemini-2.5-flash')
model.invoke('hi')


AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--fd6d427d-f019-4171-b2e9-7efb553f5548-0', usage_metadata={'input_tokens': 2, 'output_tokens': 491, 'total_tokens': 493, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 482}})

In [6]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description='Detailed feedback for the essay')
    score: int = Field(description='Score the essay out of 10', ge=0, le=10)

In [13]:
structured_model = model.with_structured_output(EvaluationSchema)

In [8]:
essay = """ Generative AI in Today's World

Generative AI (GenAI) has emerged as one of the most transformative technologies of the modern era. By enabling machines to create human-like text, images, music, and even code, GenAI is reshaping how people work, learn, and innovate. Unlike traditional AI systems that only analyze data, GenAI models actively produce new content, unlocking creativity at scale. 

In today's world, businesses leverage GenAI to automate customer support, draft reports, and accelerate product design. In education, it serves as a virtual tutor, simplifying complex concepts and personalizing learning experiences. Creative industries use it to generate art, advertisements, and storytelling ideas, while researchers employ it to explore new scientific hypotheses.

However, GenAI also raises important challenges. Concerns around misinformation, deepfakes, data privacy, and job displacement demand responsible usage. Ethical frameworks and transparency in AI systems are essential to ensure the technology benefits society as a whole.

Ultimately, GenAI represents both a tool and a responsibility. When used thoughtfully, it can enhance human potential, reduce repetitive work, and foster innovation, making it one of the most significant advancements shaping today's digital landscape.
"""

In [16]:
prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
output = structured_model.invoke(prompt)
output


EvaluationSchema(feedback='The essay is well-structured and easy to follow, using clear and concise language. It effectively covers the key aspects of Generative AI, including its applications, benefits, and challenges. The vocabulary is appropriate, and there are no noticeable grammatical errors or spelling mistakes. The flow between paragraphs is smooth, making it a highly readable and informative piece.', score=9)

In [17]:
class UPSCState(TypedDict):

    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float
